In [1]:
# flows/bitcoin_flow.py
from prefect import flow, task
import requests
import boto3
import json
from decimal import Decimal
from datetime import datetime, UTC
import time
import os
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
@task(retries=3, retry_delay_seconds=10)
def fetch_crypto_data():
    """Fetch crypto price data"""
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {
        'ids': 'bitcoin,ethereum,tether,binancecoin,solana,ripple,usd-coin,cardano,dogecoin,tron',
        'vs_currencies': 'eur',
        'include_24hr_change': 'false',
        'include_24hr_vol': 'false',
        'include_market_cap': 'false'
    }

    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

@task
def transform_data(raw_data):
    """Add timestamp and format data"""
    return {
       'timestamp': datetime.now(UTC).isoformat(),
       **{key: item['eur'] for key, item in raw_data.items()}
    }

@task
def save_to_dynamodb(data, table_name='crypto-prices'):
    """Save to DynamoDB - all cryptos in one item per timestamp"""
    from decimal import Decimal
    import time
    
    dynamodb = boto3.resource('dynamodb', region_name=os.getenv('AWS_DEFAULT_REGION'))
    table = dynamodb.Table(table_name)
    
    timestamp = data['timestamp']
    ttl = int(time.time()) + (180 * 24 * 3600)  # Expire in 180 days
    
    item = {
        'PK': 'CRYPTO_PRICES',  # fixed partition key
        'timestamp': timestamp,  # sort key
        'ttl': ttl
    }

    for key, value in data.items():
        if key == 'timestamp':
            continue
        item[key] = Decimal(str(value))
    
    table.put_item(Item=item)
    
    return 1 

@flow(name="Crypto Price Tracker")
def crypto_tracking_flow():
    """Main flow to track Crypto prices"""
    raw_data = fetch_crypto_data()
    transformed = transform_data(raw_data)
    dynamodb_path = save_to_dynamodb(transformed)

    print(f"Saved crypto data to {dynamodb_path}")



In [ ]:
if __name__ == "__main__":
    crypto_tracking_flow()

10:44:10.622 | INFO    | Flow run 'smoky-frog' - Beginning flow run 'smoky-frog' for flow 'Bitcoin Price Tracker'

10:44:10.898 | INFO    | Task run 'fetch_bitcoin_data-fd7' - Finished in state Completed()

10:44:10.912 | INFO    | Task run 'transform_data-1af' - Finished in state Completed()

10:44:11.370 | INFO    | Task run 'save_to_dynamodb-253' - Finished in state Completed()

Saved crypto data to 1


10:44:11.384 | INFO    | Flow run 'smoky-frog' - Finished in state Completed()

In [4]:
sts = boto3.client('sts')
identity = sts.get_caller_identity()

print(f"Account ID: {identity['Account']}")
print(f"User ARN: {identity['Arn']}")
print(f"Region: {boto3.Session().region_name}")

Account ID: 976095577681
User ARN: arn:aws:iam::976095577681:user/aws-practice-user
Region: eu-west-1
